In [1]:
import pandas as pd

# 讀取 CSV 資料，第一行作為標題
file_path = 'C:/Users/aaa29/台科大/台科大富邦/6669資產負債表_2025財模用.csv'  # 請替換為您的本地 CSV 檔案路徑
df = pd.read_csv(file_path, header=0)

# 取出年份行並檢查內容
year_columns = df.columns[1:].tolist()  # 假設第一列是 '年/月'，其餘列是年份

# 處理年份行中的日期格式，只保留年份（假設格式為 "13-Dec" 代表年份 2013）
year_columns = ["20" + col.split("-")[0] if int(col.split("-")[0]) <= 50 else "19" + col.split("-")[0] for col in year_columns]
print("轉換後的年份行內容:", year_columns)  # 檢查轉換後的年份行

# 更新 DataFrame 的欄位名稱
df.columns = ['年/月'] + year_columns

# 計算幾何平均成長率的函數
def calculate_geometric_growth_rate(data):
    returns = data.pct_change().dropna()  # 計算每年的成長率，並移除空值
    geometric_mean = (returns + 1).prod() ** (1 / len(returns)) - 1
    return geometric_mean

# 需要預測的項目
data_items = ['資產總額', '流動資產', '非流動資產', '負債總額', '流動負債', '非流動負債']
growth_rates = {}

# 使用幾何平均成長率計算每個項目的歷史平均成長率
for item in data_items:
    item_data = df[df['年/月'] == item]
    if not item_data.empty:
        historical_values = item_data.iloc[0, 1:]
        growth_rates[item] = calculate_geometric_growth_rate(historical_values)
    else:
        print(f"項目 '{item}' 不存在於資料中")

# 獲取最新年份，並設定預測年份
last_year = year_columns[-1]
forecast_year1 = str(int(last_year) + 1) + "F"
forecast_year2 = str(int(last_year) + 2) + "F"

# 根據成長率預測未來的數值
projections = {'年/月': data_items, forecast_year1: [], forecast_year2: []}
for item in data_items:
    item_data = df[df['年/月'] == item]
    if not item_data.empty:
        last_value = item_data.iloc[0, -1]
        growth_rate = growth_rates.get(item, 0)
        projected_1 = last_value * (1 + growth_rate)
        projected_2 = projected_1 * (1 + growth_rate)
        projections[forecast_year1].append(projected_1)
        projections[forecast_year2].append(projected_2)
    else:
        projections[forecast_year1].append(None)
        projections[forecast_year2].append(None)

# 將預測數據轉換為 DataFrame
df_projections = pd.DataFrame(projections)

# 計算「權益總額」（資產總額 - 負債總額）
if '資產總額' in df_projections['年/月'].values and '負債總額' in df_projections['年/月'].values:
    shareholders_equity_1 = df_projections.loc[df_projections['年/月'] == '資產總額', forecast_year1].values[0] - df_projections.loc[df_projections['年/月'] == '負債總額', forecast_year1].values[0]
    shareholders_equity_2 = df_projections.loc[df_projections['年/月'] == '資產總額', forecast_year2].values[0] - df_projections.loc[df_projections['年/月'] == '負債總額', forecast_year2].values[0]
    
    # 使用 pd.concat() 添加「權益總額」
    new_row = pd.DataFrame({'年/月': ['權益總額'], forecast_year1: [shareholders_equity_1], forecast_year2: [shareholders_equity_2]})
    df_projections = pd.concat([df_projections, new_row], ignore_index=True)
else:
    print("資產總額或負債總額的數據缺失，無法計算權益總額")

# 整合歷史數據與預測數據
historical_data = df[['年/月'] + year_columns].copy()
projection_data = df_projections[['年/月', forecast_year1, forecast_year2]].copy()

# 合併數據
df_final = pd.merge(historical_data, projection_data, on='年/月', how='left')

# 確保「股東權益總額」的預測值被計算並填入
for forecast_year in [forecast_year1, forecast_year2]:
    if '資產總額' in df_final['年/月'].values and '負債總額' in df_final['年/月'].values:
        # 獲取預測的資產總額和負債總額
        asset_total = df_final.loc[df_final['年/月'] == '資產總額', forecast_year].values[0]
        liability_total = df_final.loc[df_final['年/月'] == '負債總額', forecast_year].values[0]
        
        # 計算權益總額
        equity_total = asset_total - liability_total
        
        # 填入股東權益總額的欄位
        df_final.loc[df_final['年/月'] == '股東權益總額', forecast_year] = equity_total

# 設定顯示格式，不使用科學記號
pd.options.display.float_format = '{:.2f}'.format

# 輸出結果
print("完整的預測資產負債表：")
print(df_final)


轉換後的年份行內容: ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
完整的預測資產負債表：
      年/月     2014     2015      2016      2017      2018      2019      2020  \
0    資產總額  1958121  6224048  11442707  25872934  28204042  48147226  54724165   
1    流動資產  1896179  6143964  11350432  25609520  27583370  46413396  52767378   
2   非流動資產    61942    80084     92275    263414    620672   1733830   1956787   
3    負債總額  1330419  5308704   9636225  22856488  17427255  27928361  30210242   
4    流動負債  1328905  5252969   9567574  22780634  11834012  27532130  24799705   
5   非流動負債     1514    55735     68651     75854   5593243    396231   5410537   
6  股東權益總額   627702   915344   1806482   3016446  10776787  20218865  24513923   

       2021      2022      2023        2024F        2025F  
0  84737458  88837782  88845397 135744834.04 207401402.79  
1  82025003  84051090  79195299 119892087.43 181502094.33  
2   2712455   4786692   9650098  16910171.09  29632226.14  
3  5736

In [2]:
# 將結果存回本地或顯示
output_path = 'C:/Users/aaa29/台科大/台科大富邦/6669資產負債表_2025預測.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8-sig')
print("資料已儲存至:", output_path)

資料已儲存至: C:/Users/aaa29/台科大/台科大富邦/6669資產負債表_2025預測.csv


In [3]:
df_final

,年/月,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024F,2025F
0,資產總額,1958121,6224048,11442707,25872934,28204042,48147226,54724165,84737458,88837782,88845397,135744834.04,207401402.79
1,流動資產,1896179,6143964,11350432,25609520,27583370,46413396,52767378,82025003,84051090,79195299,119892087.43,181502094.33
2,非流動資產,61942,80084,92275,263414,620672,1733830,1956787,2712455,4786692,9650098,16910171.09,29632226.14
3,負債總額,1330419,5308704,9636225,22856488,17427255,27928361,30210242,57367432,50050776,46611345,69199523.06,102734087.43
4,流動負債,1328905,5252969,9567574,22780634,11834012,27532130,24799705,47165833,38562438,36494015,52732733.06,76197182.90
5,非流動負債,1514,55735,68651,75854,5593243,396231,5410537,10201599,11488338,10117330,26919020.52,71623013.77
6,股東權益總額,627702,915344,1806482,3016446,10776787,20218865,24513923,27370026,38787006,42234052,66545310.98,104667315.37


In [19]:
type(historical_values[0])

numpy.int64